In [1]:
import argparse

from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import Optimizer, nihgcn
from myutils import *
from sampler import NewSampler
from sklearn.model_selection import KFold

In [3]:
class Args:
    def __init__(self):
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}
        self.lr = 0.001  # the learning rate
        self.wd = 1e-5  # the weight decay for l2 normalizaton
        self.layer_size = [1024, 1024]  # Output sizes of every layer
        self.alpha = 0.25  # the scale for balance gcn and ni
        self.gamma = 8  # the scale for sigmod
        self.epochs = 1000  # the epochs for model


args = Args()

In [4]:
res, drug_finger, exprs, null_mask, pos_num = load_data(args)
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]

load gdsc1


In [5]:
def nihgcn_new(
    cell_exprs,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
):

    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)

    val_labels = sampler.test_data[sampler.test_mask]

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None

    model = nihgcn(
        sampler.train_data,
        cell_exprs=cell_exprs,
        drug_finger=drug_finger,
        layer_size=args.layer_size,
        alpha=args.alpha,
        gamma=args.gamma,
        device=args.device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        evaluate_fun,
        lr=args.lr,
        wd=args.wd,
        epochs=args.epochs,
        device=args.device,
    )
    true_data, predict_data = opt()
    return true_data, predict_data

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm

n_kfold = 1
n_jobs = 50  # 並列数


def process_iteration(dim, target_index, seed, args):
    """各反復処理をカプセル化した関数"""
    if dim:
        if drug_sum[target_index] < 10:
            return None, None
    else:
        if cell_sum[target_index] < 10:
            return None, None

    fold_results = []
    for fold in range(n_kfold):
        true_data, predict_data = nihgcn_new(
            cell_exprs=exprs,
            drug_finger=drug_finger,
            res_mat=res,
            null_mask=null_mask,
            target_dim=dim,
            target_index=target_index,
            evaluate_fun=roc_auc,
            args=args,
            seed=seed,
        )
        fold_results.append((true_data, predict_data))

    return fold_results


# 並列処理の実行
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()

for dim in target_dim:
    # 全タスクを事前に生成
    tasks = [
        (dim, target_index, seed, args)
        for seed, target_index in enumerate(np.arange(res.shape[dim]))
    ]

    # 並列実行（プログレスバー付き）
    results = Parallel(n_jobs=n_jobs, verbose=0, prefer="threads")(
        delayed(process_iteration)(*task)
        for task in tqdm(tasks, desc=f"Processing dim {dim}")
    )

    # 結果の結合
    for fold_results in results:
        if fold_results is None:
            continue
        for true_data, predict_data in fold_results:
            true_data_s = pd.concat(
                [true_data_s, translate_result(true_data)],
                ignore_index=True,
                copy=False,  # メモリ節約のため
            )
            predict_data_s = pd.concat(
                [predict_data_s, translate_result(predict_data)],
                ignore_index=True,
                copy=False,
            )

  0%|          | 0/916 [00:00<?, ?it/s]

epoch:   0 loss:0.699444 auc:0.5786
epoch:  20 loss:0.198788 auc:0.9873
epoch:  40 loss:0.179393 auc:0.9900
epoch:  60 loss:0.165992 auc:0.9881
epoch:  80 loss:0.156055 auc:0.9885
epoch: 100 loss:0.146204 auc:0.9842
epoch: 120 loss:0.136821 auc:0.9839
epoch: 140 loss:0.129492 auc:0.9869
epoch: 160 loss:0.123500 auc:0.9873
epoch: 180 loss:0.120722 auc:0.9873
epoch: 200 loss:0.118528 auc:0.9885
epoch: 220 loss:0.115119 auc:0.9869
epoch: 240 loss:0.112930 auc:0.9873
epoch: 260 loss:0.113125 auc:0.9862
epoch: 280 loss:0.110879 auc:0.9877
epoch: 300 loss:0.109857 auc:0.9889
epoch: 320 loss:0.110196 auc:0.9881
epoch: 340 loss:0.108717 auc:0.9885
epoch: 360 loss:0.108369 auc:0.9892
epoch: 380 loss:0.107977 auc:0.9892
epoch: 400 loss:0.108226 auc:0.9896
epoch: 420 loss:0.107371 auc:0.9896
epoch: 440 loss:0.106817 auc:0.9896
epoch: 460 loss:0.107054 auc:0.9896
epoch: 480 loss:0.110518 auc:0.9915
epoch: 500 loss:0.106384 auc:0.9892
epoch: 520 loss:0.106514 auc:0.9892
epoch: 540 loss:0.105991 auc

  0%|          | 1/916 [00:41<10:39:32, 41.94s/it]

Fit finished.
epoch:   0 loss:0.698390 auc:0.5220
epoch:  20 loss:0.197045 auc:0.9666
epoch:  40 loss:0.177233 auc:0.9729
epoch:  60 loss:0.164274 auc:0.9765
epoch:  80 loss:0.154799 auc:0.9782
epoch: 100 loss:0.145052 auc:0.9762
epoch: 120 loss:0.135634 auc:0.9749
epoch: 140 loss:0.129494 auc:0.9716
epoch: 160 loss:0.123025 auc:0.9739
epoch: 180 loss:0.119772 auc:0.9742
epoch: 200 loss:0.118807 auc:0.9706
epoch: 220 loss:0.114452 auc:0.9739
epoch: 240 loss:0.114696 auc:0.9759
epoch: 260 loss:0.111729 auc:0.9752
epoch: 280 loss:0.110646 auc:0.9769
epoch: 300 loss:0.109549 auc:0.9745
epoch: 320 loss:0.112645 auc:0.9696
epoch: 340 loss:0.108535 auc:0.9736
epoch: 360 loss:0.108891 auc:0.9769
epoch: 380 loss:0.108818 auc:0.9759
epoch: 400 loss:0.107406 auc:0.9745
epoch: 420 loss:0.107535 auc:0.9752
epoch: 440 loss:0.107803 auc:0.9762
epoch: 460 loss:0.107065 auc:0.9745
epoch: 480 loss:0.106394 auc:0.9752
epoch: 500 loss:0.107461 auc:0.9772
epoch: 520 loss:0.105766 auc:0.9772
epoch: 540 los

  0%|          | 2/916 [01:22<10:22:56, 40.89s/it]

...

Fit finished.
epoch:   0 loss:0.709807 auc:0.4831
epoch:  20 loss:0.197913 auc:0.9786
epoch:  40 loss:0.177989 auc:0.9836
epoch:  60 loss:0.165154 auc:0.9804
epoch:  80 loss:0.155421 auc:0.9790
epoch: 100 loss:0.146188 auc:0.9765
epoch: 120 loss:0.138243 auc:0.9726
epoch: 140 loss:0.129419 auc:0.9754
epoch: 160 loss:0.124381 auc:0.9751
epoch: 180 loss:0.120488 auc:0.9783
epoch: 200 loss:0.116577 auc:0.9783
epoch: 220 loss:0.115272 auc:0.9779
epoch: 240 loss:0.112831 auc:0.9790
epoch: 260 loss:0.113827 auc:0.9772
epoch: 280 loss:0.110771 auc:0.9769
epoch: 300 loss:0.110676 auc:0.9779
epoch: 320 loss:0.108994 auc:0.9758
epoch: 340 loss:0.116165 auc:0.9765
epoch: 360 loss:0.108455 auc:0.9776
epoch: 380 loss:0.107572 auc:0.9765
epoch: 400 loss:0.107115 auc:0.9761
epoch: 420 loss:0.107151 auc:0.9761
epoch: 440 loss:0.108274 auc:0.9779
epoch: 460 loss:0.106564 auc:0.9754
epoch: 480 loss:0.106017 auc:0.9751
epoch: 500 loss:0.107630 auc:0.9790
epoch: 520 loss:0.105868 auc:0.9751
epoch: 540 los

  3%|▎         | 30/916 [29:02<14:47:54, 60.13s/it]

Fit finished.
epoch:   0 loss:0.697179 auc:0.4230
epoch:  20 loss:0.198291 auc:0.9646
epoch:  40 loss:0.177978 auc:0.9716
epoch:  60 loss:0.164898 auc:0.9768
epoch:  80 loss:0.155517 auc:0.9792
epoch: 100 loss:0.146053 auc:0.9792
epoch: 120 loss:0.137118 auc:0.9789
epoch: 140 loss:0.129594 auc:0.9797
epoch: 160 loss:0.123786 auc:0.9823
epoch: 180 loss:0.119376 auc:0.9820
epoch: 200 loss:0.118248 auc:0.9839
epoch: 220 loss:0.114326 auc:0.9826
epoch: 240 loss:0.115563 auc:0.9787
epoch: 260 loss:0.111736 auc:0.9818
epoch: 280 loss:0.110275 auc:0.9826
epoch: 300 loss:0.111269 auc:0.9870
epoch: 320 loss:0.109000 auc:0.9860
epoch: 340 loss:0.110724 auc:0.9831
epoch: 360 loss:0.107911 auc:0.9865
epoch: 380 loss:0.107648 auc:0.9836
epoch: 400 loss:0.107803 auc:0.9870
epoch: 420 loss:0.106751 auc:0.9862


  3%|▎         | 30/916 [29:29<14:31:09, 59.00s/it]


KeyboardInterrupt: 

In [ ]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")